## Setup: Install Dependencies
Run this once to load project libs (requests, pandas, etc.).

In [1]:
# Install from requirements.txt (run once)
%pip install -r ../requirements.txt
print('✅ Deps installed—restart kernel if needed (Kernel > Restart).')

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
✅ Deps installed—restart kernel if needed (Kernel > Restart).



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\kumar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# AIOps Anomaly Detection: Interactive Demo

**Overview**: Offline simulation of automated anomaly detection for IT ops (e.g., Azure VM CPU metrics). 
- Learns 'normal' patterns from time-series data.
- Flags deviations without manual thresholds.
- Visualizes actual vs. expected for proactive downtime prevention.

**Tune & Re-run**: Change `anomaly_prob` or `num_points` in Cell 3, then re-execute.

Future: Replace mocks with Azure Monitor fetches for real data.

In [2]:
# Imports (path to root for src/)
import sys
import random
sys.path.append('..')
from src.detect_anomalies import create_payload_with_anomalies
from src.visualize_anomalies import plot_detection
from datetime import datetime

In [3]:
# Step 1: Generate synthetic time-series data
# Simulates metrics like CPU % with patterns + random anomalies (spikes/drops)
random.seed(42)  # Reproducible for demos
payload = create_payload_with_anomalies(
    num_points=50,  # e.g., 50 mins of data
    anomaly_prob=0.15,  # 15% deviation chance (tune here!)
    base_value=75  # Normal baseline
)
print(f"Generated {len(payload['series'])} points with ~{int(0.15*50)} expected anomalies.")
print("Sample data:", payload['series'][:3])  # Peek at first points

Generated 50 points with ~7 expected anomalies.
Sample data: [{'timestamp': '2025-10-08T17:56:26.221730', 'value': 101.89485435196121}, {'timestamp': '2025-10-08T17:55:26.221730', 'value': 77.76699487422911}, {'timestamp': '2025-10-08T17:54:26.221730', 'value': 72.86938832629416}]


In [4]:
# Step 2: Mock Detection (AIOps 'learns' patterns & flags deviations)
# Real: Call Azure Anomaly Detector API with env vars
result = {
    'isAnomaly': True,  # Overall flag
    'expectedValues': [75] * len(payload['series']),  # Mock learned baseline
    'anomalyStatus': [1 if random.random() < 0.15 else 0 for _ in payload['series']]  # Per-point flags
}
anomaly_count = sum(result['anomalyStatus'])
print(f"Mock detection: {anomaly_count} anomalies flagged! 🚨\n(Proactive alert: Auto-scale resources?)")

Mock detection: 9 anomalies flagged! 🚨
(Proactive alert: Auto-scale resources?)


In [5]:
# Step 3: Visualize (actual vs. expected, red dots on deviations)
plot_detection(payload, result, save_path='anomaly_demo_nb.png', show_plot=True)
print("Plot rendered—check inline/PNG for spikes & reds!")

📊 Plot saved to anomaly_demo_nb.png
Plot rendered—check inline/PNG for spikes & reds!


In [6]:
# Step 4: Mock Alerting (Proactive Response on Anomalies)
# Real: Post to Azure Action Group webhook or email
alert_threshold = 3  # e.g., alert if >3 anomalies in window
if anomaly_count > alert_threshold:
    alert_msg = f"""
🚨 AIOps Alert: High Deviation Detected!
- Metrics: CPU % (simulated)
- Anomalies: {anomaly_count} in last {len(payload['series'])} mins
- Top Deviation: Max spike {max([p['value'] for p in payload['series']])}% at ~{payload['series'][0]['timestamp']}
- Action: Auto-scale resources or investigate logs.
- Severity: Medium (tune for multi-cloud)
"""
    print(alert_msg)
    
    # Mock 'send' (e.g., to email/Slack)
    print("📧 Mock: Alert sent to Ops team via Azure Alerts/Teams.")
else:
    print("✅ No alerts triggered—all within normal patterns.")


🚨 AIOps Alert: High Deviation Detected!
- Metrics: CPU % (simulated)
- Anomalies: 9 in last 50 mins
- Top Deviation: Max spike 111.84357447768423% at ~2025-10-08T17:56:26.221730
- Action: Auto-scale resources or investigate logs.
- Severity: Medium (tune for multi-cloud)

📧 Mock: Alert sent to Ops team via Azure Alerts/Teams.


## Multi-Cloud Extension + Alerting

| Cloud | Data Source | Alert Integration |
|-------|-------------|-------------------|
| **Azure** | Monitor/Log Analytics | Action Groups + Logic Apps for emails/SMS. |
| **AWS** | CloudWatch | SNS notifications on alarms. |
| **GCP** | Operations Suite | Alerting policies to Pub/Sub. |

**Next Steps**: 
- Wire mocks to real webhooks.
- Test with simulated downtime (e.g., anomaly_prob=0.5).